# Script summay
I aim to uncover stocks that represent a higher risk, thus guiding investors away from potential losses. This comprehensive process is designed to identify and evaluate companies from underperforming industries or those exhibiting signs of financial distress.

In [1]:
# Import necessary libraries
import warnings
import numpy as np
import requests
from bs4 import BeautifulSoup
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta


warnings.filterwarnings('ignore')

# Set pandas display options for easier DataFrame inspection
pd.set_option('display.max_rows', None)  
pd.set_option('display.max_columns', None)  

## Industry Performance Review
### Fetching S&P 500/ Russell 1000 Companies and Industries
The process begins by extracting a list of S&P 500/Russell 1000 companies and their respective industries from Wikipedia. This step is crucial for understanding the broader market landscape and setting the stage for a more granular analysis.

In [ ]:
# Fetch russell 1000 companies and their industries from Wikipedia
url = "https://en.wikipedia.org/wiki/Russell_1000_Index"
page = requests.get(url)
soup = BeautifulSoup(page.content, "html.parser")

# Extract tickers and industries
table = soup.find('table', {'class': 'wikitable sortable'})
tickers = []
industries = []
company_names = []
for row in table.findAll('tr')[1:]:
    company_name = row.find_all('td')[0].text.strip()
    ticker = row.find_all('td')[1].text.strip()
    industry = row.find_all('td')[2].text.strip()
    company_names.append(company_name)
    tickers.append(ticker)
    industries.append(industry)

# Combine tickers and industries into a DataFrame
df_russell1000 = pd.DataFrame({
    'Company': company_names,
    'Ticker': tickers,
    'Industry': industries
})

# Fetch historical performance data for each company
end_date = datetime.today()
start_date = end_date - timedelta(days=250)

In [ ]:
# Fetch the Wikipedia page and extract S&P 500 companies and their industries
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
page = requests.get(url)
soup = BeautifulSoup(page.content, "html.parser")

# Extract tickers and industries
table = soup.find('table', {'class': 'wikitable sortable'})
tickers = []
industries = []
company_names = []
for row in table.findAll('tr')[1:]:
    company_name = row.find_all('td')[1].text.strip()
    ticker = row.find_all('td')[0].text.strip()
    industry = row.find_all('td')[2].text.strip()
    company_names.append(company_name)
    tickers.append(ticker)
    industries.append(industry)

# Combine tickers and industries into a DataFrame
df_SP500 = pd.DataFrame({
    'Company': company_names,
    'Ticker': tickers,
    'Industry': industries
})


# Fetch historical performance data for each company
end_date = datetime.today()
start_date = end_date - timedelta(days=250)


### Identifying the Worst-Performing Industry

Using the stock price data for the past ~250 days, we calculate the percentage change in stock prices for each company. 

Aggregating these changes by industry allows us to identify the sectors with the worst average performance, signaling industries that may be experiencing systemic challenges.

In [2]:


# Function to fetch historical data
def fetch_performance(ticker):
    stock = yf.Ticker(ticker)
    hist = stock.history(start=start_date, end=end_date)
    if not hist.empty:
        return (hist.iloc[-1]['Close'] - hist.iloc[0]['Close']) / hist.iloc[0]['Close']
    else:
        return None
performance = df_russell1000.copy()

# Add performance data to the DataFrame
performance['Performance'] = df_russell1000['Ticker'].apply(fetch_performance)

# Aggregate performance by industry and sort
industry_performance = performance.groupby('Industry')['Performance'].mean().sort_values()

# Convert the sorted Series to a DataFrame for better display
df_industry_performance_russell1000 = industry_performance.reset_index()
df_industry_performance_russell1000.columns = ['Industry', 'Average Performance']


df_industry_performance_negative = df_industry_performance_russell1000[df_industry_performance_russell1000['Average Performance'] <0]
# Display the DataFrame
display(df_industry_performance_russell1000)

#  Identify the worst-performing industry
worst_performing_industry = industry_performance.idxmin()

print(f"The worst-performing industry in the Russell 1000 over the last year is: {worst_performing_industry}")


BRK.B: No timezone found, symbol may be delisted
BF.A: No timezone found, symbol may be delisted
BF.B: No price data found, symbol may be delisted (1d 2023-08-04 18:02:51.846013 -> 2024-04-10 18:02:51.846013)
CWEN.A: No timezone found, symbol may be delisted
HEI.A: No timezone found, symbol may be delisted
LEN.B: No price data found, symbol may be delisted (1d 2023-08-04 18:02:51.846013 -> 2024-04-10 18:02:51.846013)
UHAL.B: No price data found, symbol may be delisted (1d 2023-08-04 18:02:51.846013 -> 2024-04-10 18:02:51.846013)


,Industry,Average Performance
0,Communication Services,0.017334
1,Consumer Staples,0.022661
2,Real Estate,0.026607
3,Health Care,0.035049
4,Utilities,0.061626
5,Consumer Discretionary,0.071208
6,Materials,0.095571
7,Information Technology,0.149873
8,Industrials,0.153868
9,Energy,0.156617


The worst-performing industry in the Russell 1000 over the last year is: Communication Services


In [3]:

# Function to fetch historical data
def fetch_performance(ticker):
    stock = yf.Ticker(ticker)
    hist = stock.history(start=start_date, end=end_date)
    if not hist.empty:
        return (hist.iloc[-1]['Close'] - hist.iloc[0]['Close']) / hist.iloc[0]['Close']
    else:
        return None
performance = df_SP500.copy()

# Add performance data to the DataFrame
performance['Performance'] = df_SP500['Ticker'].apply(fetch_performance)

# Aggregate performance by industry and sort
industry_performance = performance.groupby('Industry')['Performance'].mean().sort_values()

# Convert the sorted Series to a DataFrame for better display
df_industry_performance = industry_performance.reset_index()
df_industry_performance.columns = ['Industry', 'Average Performance']


df_industry_performance_negative = df_industry_performance[df_industry_performance['Average Performance'] <0]
# Display the DataFrame
display(df_industry_performance)

# Identify the worst-performing industry
worst_performing_industry = industry_performance.idxmin()

print(f"The worst-performing industry in the S&P 500 over the last year is: {worst_performing_industry}")


BRK.B: No timezone found, symbol may be delisted
BF.B: No price data found, symbol may be delisted (1d 2023-08-04 18:13:30.207947 -> 2024-04-10 18:13:30.207947)


,Industry,Average Performance
0,Consumer Staples,-0.022416
1,Real Estate,0.035147
2,Health Care,0.052318
3,Utilities,0.058295
4,Communication Services,0.092665
5,Consumer Discretionary,0.093096
6,Materials,0.105887
7,Energy,0.164291
8,Industrials,0.166002
9,Financials,0.171324


The worst-performing industry in the S&P 500 over the last year is: Consumer Staples


In [4]:
# Copy the performance DataFrame to a new one for YoY calculations
df_yoyperformance = performance.copy()

# Add placeholder columns for YoY financial data
financial_columns = [
    'YoY Revenue 2023-2022', 'YoY Revenue 2022-2021', 'YoY Revenue 2021-2020',
    'YoY Profit 2023-2022', 'YoY Profit 2022-2021', 'YoY Profit 2021-2020'
]
for column in financial_columns:
    df_yoyperformance[column] = None  # Initialize with None for later calculation

def calculate_yoy_change(current, previous):
    """
    Calculates Year-over-Year change as a percentage.
    
    :param current: Current year's value
    :param previous: Previous year's value
    :return: YoY change as a percentage or None if inputs are not valid
    """
    if previous and current:
        return (current - previous) / previous
    return None

# Iterate through the DataFrame to fetch financial data for each ticker and calculate YoY changes
for index, row in df_yoyperformance.iterrows():
    ticker = row['Ticker']
    try:
        stock = yf.Ticker(ticker)
        financials = stock.financials
        if financials.empty:
            print(f"No financial data for {ticker}")
            continue

        # Fetch revenue and net income data
        revenue = financials.loc['Total Revenue']
        net_income = financials.loc['Net Income']

        # Calculate YoY changes for both revenue and profit, assuming at least 4 years of data available
        if len(revenue) >= 4 and len(net_income) >= 4:
            for i in range(3):  # Loop over the three most recent years
                df_yoyperformance.at[index, financial_columns[i]] = calculate_yoy_change(revenue[i], revenue[i+1])
                df_yoyperformance.at[index, financial_columns[i+3]] = calculate_yoy_change(net_income[i], net_income[i+1])
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")


No financial data for BRK.B
No financial data for BF.B


## Financial Health Metrics

Next, apply a series of financial health indicators — including Current and Quick Ratios for liquidity, P/E Ratio for valuation, Debt Leverage, and the Z-Score for bankruptcy likelihood — to further evaluate companies in underperforming sectors. These indicators help highlight firms that may be encountering financial difficulties.  

- **A Current Ratio below the industry average**: `Current Assets / Current Liabilities`  
  This indicates a potential liquidity problem, suggesting the company might struggle to meet short-term obligations.
- **A Quick Ratio below the industry average**: `(Current Assets - Inventory) / Current Liabilities`  
  Similar to the Current Ratio, a low Quick Ratio points to difficulties in covering immediate liabilities, emphasizing liquidity concerns.
- **A P/E Ratio above the industry average**: `Market Price per Share / Earnings per Share (EPS)`  
  An elevated P/E ratio can signal overvaluation, particularly if earnings do not justify the high share price.
- **A Debt Leverage above the industry average**: `Total Debt / Shareholders' Equity`  
  High debt levels compared to equity can indicate financial risk, especially if the industry is facing downturns.
- **A Z-Score below 1.8**: A Z-Score below 1.8 suggests a high risk of bankruptcy.
  - Formula: `1.2(A) + 1.4(B) + 3.3(C) + 0.6(D) + 1.0(E)`
    - Where:
      - A = Working Capital / Total Assets
      - B = Retained Earnings / Total Assets
      - C = Earnings Before Interest and Taxes (EBIT) / Total Assets
      - D = Market Value of Equity / Total Liabilities
      - E = Sales / Total Assets


In [5]:
# Function to fetch and calculate key financial ratios for a given ticker symbol
def fetch_financial_ratios(ticker):
    try:
        stock = yf.Ticker(ticker)
        balance_sheet_annual = stock.balance_sheet
        financials_annual = stock.financials
        
        # Initialize ratio variables
        current_ratio = np.nan
        quick_ratio = np.nan
        pe_ratio = np.nan
        debt_leverage = np.nan
        z_score = np.nan

        # Current Ratio
        try:
            current_assets = balance_sheet_annual.loc['Current Assets'][0]
            current_liabilities = balance_sheet_annual.loc['Current Liabilities'][0]
            current_ratio = current_assets / current_liabilities
        except Exception as e:
            pass
            # print(f"Error calculating current ratio for {ticker}: {e}")
        
        # Quick Ratio
        try:
            inventory = balance_sheet_annual.loc['Inventory'][0] if 'Inventory' in balance_sheet_annual.index else 0
            quick_ratio = (current_assets - inventory) / current_liabilities
        except Exception as e:
            pass
            # print(f"Error calculating quick ratio for {ticker}: {e}")
        
        # PE Ratio
        try:
            pe_ratio = stock.history_metadata['regularMarketPrice']/ financials_annual.loc["Diluted EPS"][0]
        except Exception as e:
            pass
            # print(f"Error calculating PE ratio for {ticker}: {e}")

        # Debt Leverage
        try:
            total_assets = balance_sheet_annual.loc['Total Assets'][0]
            total_liabilities_net_minority_interest = balance_sheet_annual.loc['Total Liabilities Net Minority Interest'][0]
            total_equity = total_assets - total_liabilities_net_minority_interest
            long_term_debt = balance_sheet_annual.loc['Long Term Debt'][0]
            debt_leverage = long_term_debt / total_equity
        except Exception as e:
            pass
            # print(f"Error calculating debt leverage for {ticker}: {e}")

        # Z-Score (Example Calculation)
        try:
            current_assets = balance_sheet_annual.loc['Current Assets'][0]
            current_liabilities = balance_sheet_annual.loc['Current Liabilities'][0]
            total_assets = balance_sheet_annual.loc['Total Assets'][0]
            total_liabilities_net_minority_interest = balance_sheet_annual.loc['Total Liabilities Net Minority Interest'][0]
            retained_earnings = balance_sheet_annual.loc['Retained Earnings'][0]
            ebit = financials_annual.loc['EBIT'][0]
            total_revenue = financials_annual.loc['Operating Revenue'][0]
            market_value_equity = stock.history_metadata['regularMarketPrice'] * stock.get_shares_full(start="2024-01-01", end=None)[-1]
    
            A = (current_assets - current_liabilities) / total_assets if total_assets else np.nan
            B = retained_earnings / total_assets if total_assets else np.nan
            C = ebit / total_assets if total_assets else np.nan
            D = market_value_equity / total_liabilities_net_minority_interest if total_liabilities_net_minority_interest else np.nan
            E = total_revenue / total_assets if total_assets else np.nan
            z_score = 1.2*A + 1.4*B + 3.3*C + 0.6*D + 1.0*E
        except Exception as e:
            pass
            # print(f"Error calculating Z-Score for {ticker}: {e}")
        
        return current_ratio, quick_ratio, pe_ratio, debt_leverage, z_score
    except Exception as e:
            # print(f"Could not fetch data for {ticker}: {e}")
        return np.nan, np.nan, np.nan, np.nan, np.nan

df_ratio = df_yoyperformance.copy()
# Apply the function to each ticker and update the DataFrame
df_ratio['Current Ratio'], df_ratio['Quick Ratio'], df_ratio['PE Ratio'], df_ratio['Debt Leverage'], df_ratio['Z-Score'] = zip(*df_ratio['Ticker'].apply(fetch_financial_ratios))

display(df_ratio.head())


BRK.B: No data found, symbol may be delisted
BF.B: Period '1wk' is invalid, must be one of ['1mo', '3mo', '6mo', 'ytd', '1y', '2y', '5y', '10y', 'max']


,Company,Ticker,Industry,Performance,YoY Revenue 2023-2022,YoY Revenue 2022-2021,YoY Revenue 2021-2020,YoY Profit 2023-2022,YoY Profit 2022-2021,YoY Profit 2021-2020,Current Ratio,Quick Ratio,PE Ratio,Debt Leverage,Z-Score
0,3M,MMM,Industrials,0.110718,-0.045225,-0.031848,0.098527,-2.210836,-0.02432,0.086621,1.070733,0.755508,-7.334125,2.688578,1.811349
1,A. O. Smith,AOS,Industrials,0.187688,0.026346,0.060753,0.222291,1.361476,-0.516116,0.412293,1.587115,1.060933,NaN,0.063598,9.101499
2,Abbott,ABT,Health Care,0.043432,-0.081186,0.013418,0.244654,-0.174528,-0.019516,0.573081,1.637887,1.163211,34.101227,0.350246,5.105538
3,AbbVie,ABBV,Health Care,0.166676,-0.064354,0.033044,0.226902,-0.589135,0.025472,1.500433,0.872123,0.763801,62.143382,5.020102,2.001841
4,Accenture,ACN,Information Technology,0.047868,0.040871,0.218883,0.140013,-0.000816,0.164278,0.15642,1.298344,1.298344,30.120706,0.001629,7.433355


## Financial Ratios Analysis

For companies within the lagging industries, we calculate the aforementioned financial ratios. By comparing these metrics against industry averages, we can pinpoint firms that are at a greater risk due to poor liquidity, high valuation, excessive debt, or a higher potential for bankruptcy.

## In-depth Filtering and Analysis

Apply strict criteria to further refine the list of companies:
- A Current Ratio less than 1 indicates immediate liquidity concerns.
- A Z-Score below 1.8 signals a high risk of bankruptcy.
- Debt Leverage greater than 2 suggests substantial financial risk due to over-leverage.

In [6]:

for industry in list(df_industry_performance["Industry"]):
    # Filter df_ratio for companies belonging to the current industry
    industry_df = df_ratio[df_ratio['Industry'] == industry].sort_values(by='Z-Score', ascending=True)
    
    # Calculate mean financial ratios for the industry
    mean_ratios = industry_df[['Current Ratio', 'Quick Ratio', 'PE Ratio', 'Debt Leverage', 'Z-Score']].mean()
    
    # Print the mean values for the current industry
    print(f"Mean ratios for {industry}:")
    print(f"Current Ratio: {mean_ratios['Current Ratio']:.2f}, Quick Ratio: {mean_ratios['Quick Ratio']:.2f}, " +
          f"PE Ratio: {mean_ratios['PE Ratio']:.2f}, Debt Leverage: {mean_ratios['Debt Leverage']:.2f}, " +
          f"Z-Score: {mean_ratios['Z-Score']:.2f}")
    
    # Identify companies with concerning financial health based on defined conditions
    filtered_companies = industry_df[
        (industry_df['Current Ratio'] < mean_ratios['Current Ratio']) &
        (industry_df['Quick Ratio'] < mean_ratios['Quick Ratio']) &
        (industry_df['PE Ratio'] > mean_ratios['PE Ratio']) &
        (industry_df['Debt Leverage'] > mean_ratios['Debt Leverage']) &
        (industry_df['Current Ratio'] < 1) &  
        (industry_df['Z-Score'] < 1.8) &  
        (industry_df['Debt Leverage'] > 2) 
    ]
    
    # Display companies with worse than average ratios
    if not filtered_companies.empty:
        print(f"Companies in {industry} with worse than average ratios:")
        display(filtered_companies)
    else:
        print(f"No companies in {industry} with worse than average ratios.\n")

Mean ratios for Consumer Staples:
Current Ratio: 1.11, Quick Ratio: 0.64, PE Ratio: -64.58, Debt Leverage: 1.09, Z-Score: 4.20
No companies in Consumer Staples with worse than average ratios.

Mean ratios for Real Estate:
Current Ratio: 2.65, Quick Ratio: 2.65, PE Ratio: 31.30, Debt Leverage: 2.00, Z-Score: 2.21
Companies in Real Estate with worse than average ratios:


,Company,Ticker,Industry,Performance,YoY Revenue 2023-2022,YoY Revenue 2022-2021,YoY Revenue 2021-2020,YoY Profit 2023-2022,YoY Profit 2022-2021,YoY Profit 2021-2020,Current Ratio,Quick Ratio,PE Ratio,Debt Leverage,Z-Score
263,Iron Mountain,IRM,Real Estate,0.277904,0.073814,0.136266,0.083009,-0.669227,0.237133,0.313768,0.776118,0.776118,120.142857,30.310223,0.895335
29,American Tower,AMT,Real Estate,0.002620,0.040435,0.144727,0.163576,-0.159984,-0.312303,0.51881,0.511774,0.511774,57.220126,3.288788,1.096212


Mean ratios for Health Care:
Current Ratio: 2.02, Quick Ratio: 1.56, PE Ratio: 54.79, Debt Leverage: 1.34, Z-Score: 5.89
No companies in Health Care with worse than average ratios.

Mean ratios for Utilities:
Current Ratio: 0.74, Quick Ratio: 0.59, PE Ratio: 17.92, Debt Leverage: 1.50, Z-Score: 0.90
Companies in Utilities with worse than average ratios:


,Company,Ticker,Industry,Performance,YoY Revenue 2023-2022,YoY Revenue 2022-2021,YoY Revenue 2021-2020,YoY Profit 2023-2022,YoY Profit 2022-2021,YoY Profit 2021-2020,Current Ratio,Quick Ratio,PE Ratio,Debt Leverage,Z-Score
199,FirstEnergy,FE,Utilities,0.066232,0.032988,0.119206,0.031696,1.714286,-0.683554,0.189064,0.476792,0.38173,19.708333,2.096464,0.697023


Mean ratios for Communication Services:
Current Ratio: 1.28, Quick Ratio: 1.18, PE Ratio: 30.46, Debt Leverage: -8.17, Z-Score: 3.39
No companies in Communication Services with worse than average ratios.

Mean ratios for Consumer Discretionary:
Current Ratio: 2.28, Quick Ratio: 1.28, PE Ratio: 16.61, Debt Leverage: 1.23, Z-Score: 4.63
Companies in Consumer Discretionary with worse than average ratios:


,Company,Ticker,Industry,Performance,YoY Revenue 2023-2022,YoY Revenue 2022-2021,YoY Revenue 2021-2020,YoY Profit 2023-2022,YoY Profit 2022-2021,YoY Profit 2021-2020,Current Ratio,Quick Ratio,PE Ratio,Debt Leverage,Z-Score
346,Norwegian Cruise Line Holdings,NCLH,Consumer Discretionary,0.025070,0.765142,6.4751,-0.493725,-1.073209,-0.496313,0.123133,0.217455,0.191349,47.179487,40.937036,0.047777
408,Royal Caribbean Group,RCL,Consumer Discretionary,0.259623,0.572398,4.770235,-0.306412,-1.787106,-0.590114,-0.092706,0.190618,0.164238,20.744849,4.027761,1.093869
186,Expedia Group,EXPE,Consumer Discretionary,0.273379,0.100454,0.356943,0.65378,1.264205,28.333333,-1.004594,0.783332,0.783332,24.745763,2.244436,1.173229


Mean ratios for Materials:
Current Ratio: 1.83, Quick Ratio: 1.24, PE Ratio: 19.87, Debt Leverage: 0.73, Z-Score: 2.99
No companies in Materials with worse than average ratios.

Mean ratios for Energy:
Current Ratio: 1.17, Quick Ratio: 0.95, PE Ratio: 13.72, Debt Leverage: 0.66, Z-Score: 3.03
No companies in Energy with worse than average ratios.

Mean ratios for Industrials:
Current Ratio: 1.60, Quick Ratio: 1.24, PE Ratio: 28.99, Debt Leverage: 0.98, Z-Score: 5.45
No companies in Industrials with worse than average ratios.

Mean ratios for Financials:
Current Ratio: 2.08, Quick Ratio: 2.08, PE Ratio: 17.33, Debt Leverage: 0.45, Z-Score: 3.79
No companies in Financials with worse than average ratios.

Mean ratios for Information Technology:
Current Ratio: 1.98, Quick Ratio: 1.59, PE Ratio: 48.50, Debt Leverage: 1.35, Z-Score: 8.28
No companies in Information Technology with worse than average ratios.

